In [ ]:
# get current directory
import sys
import os 
sys.path.append(os.path.dirname(os.getcwd()))

from langchain.chat_models import ChatAnthropic
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chat_models import ChatAnthropic
from langchain.retrievers import AmazonKendraRetriever
from langchain.chains import ConversationalRetrievalChain
from datetime import datetime
from conversations.prompt import STANDALONE_PROMPT

ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
llm = ChatAnthropic(anthropic_api_key=ANTHROPIC_API_KEY)
retriever = AmazonKendraRetriever(index_id="2e02c2d0-5223-43f5-bd51-5423bb74aa1f", region_name="eu-west-2")

In [ ]:
SUFFIX = """
Adviser: Here are a few documents in <documents> tags:
<documents>
{context}
</documents>
Based on the above documents, provide a detailed answer for, {question}. Be concise in your response and make sure to include reference to any location names \
stated in the question, and make sure your answer is relevant to the laws and rules of the location specified in the question. Also make sure to add a "Sources:" section \
at the end of your answer. The "Sources:" section should cite the url of the source documents you used in your answer. \
ONLY PUT CITED DOCUMENTS IN THE "Sources:" SECTION AND NO WHERE ELSE IN YOUR RESPONSE. IT IS CRUCIAL that citations only happens in the "Sources:" section. \
DO NOT INCLUDE ANY DOCUMENTS IN THE "Sources:" THAT YOU DID NOT USE IN YOUR RESPONSE.
Example: "Sources: <a href"https://www.citizensadvice.org.uk/debt-and-money/help-with-debt/">https://www.citizensadvice.org.uk/debt-and-money/help-with-debt/</a>"

If the question discusses 'my client', your answer should refer to 'your client'.

If information is needed to definitively answer the question, phrase this as a step by step list of questions that the adviser should ask the client and use language like 'could be' instead if 'is'. In the list of questions, use simple language.

".

Use <b>bold</b> to highlight the most question relevant parts in your response.

Caddy:
"""

CORE_PROMPT = PromptTemplate.from_template(SUFFIX)

In [ ]:
memory = ConversationEntityMemory(llm=llm)
history = []
prompt = "How do I claim UC?"
chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    # condense_question_prompt={"prompt": STANDALONE_PROMPT},
    # condense_question_prompt=STANDALONE_PROMPT,
    # return_source_documents=True,
    # return_generated_question=True,
    # combine_docs_chain_kwargs={"prompt": CORE_PROMPT},
    # combine_docs_chain_kwargs=CORE_PROMPT,
    memory=memory
)

# chain = prompt | llm

In [ ]:
chain({'foo': 1, 'bar': 2})

In [ ]:
prompt = "How do I claim UC?"
history = []
# ai_response = chain({"question": prompt})
# ai_response = chain({"question": prompt, "chat_history": history})
chain.invoke({"question": prompt, "chat_history": history})

In [ ]:
response = chain.invoke({"prompt": "How does one eat a chicken"})
response

In [ ]:
response.content

In [ ]:
chain.predict(
    input="""
    John helped Max study for a math exam. 
    Thanks to him, Max got a high grade from the exam.
    """)